# Data cleaning and merging dataframes

## Loading multiple datasets

### Google way

In [1213]:
import warnings; 
warnings.simplefilter('ignore')

In [1214]:
import pandas as pd

# orderlines.csv
url = 'https://drive.google.com/file/d/14Y7g5ITyf6LMyPoKc9wr010V9StaCUux/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orderlines = pd.read_csv(path)

# orders.csv
url = 'https://drive.google.com/file/d/1BLEHcP-9fm9Rv7A01H3co2XBMnSr66YC/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orders = pd.read_csv(path)

# brands.csv
url = 'https://drive.google.com/file/d/1BrNrIY0F1LbyXtyaQygUBXVxQGB3JBqx/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
brands = pd.read_csv(path)

# products.csv
url = 'https://drive.google.com/file/d/1UfsHI80cpQqGfsH2g4T4Tsw8cWayOfzC/view?usp=sharing' 
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
products = pd.read_csv(path)

df_list = [orderlines, orders, brands, products]
files = ['orderlines','orders','brands','products']

### Classical way

Reading file by file

In [1215]:
# import pandas as pd

# path = '../data/eniac/'
# orderlines = pd.read_csv(path + 'orderlines.csv')
# orders = pd.read_csv(path + 'orders.csv')
# brands = pd.read_csv(path + 'brands.csv')
# products = pd.read_csv(path + 'products.csv')

# df_list = [orderlines, orders, brands, products]
# files = ['orderlines','orders','brands','products']

### Another way

Using `os` and a loop to read all the files from a directory. It's also possible to read only files with a certain extension (like `.csv`):

In [1216]:
# import pandas as pd
# import os
# path = '../data/eniac/'
# path, dirs, files = next(os.walk(path))
# #print(files)

# # remove non-csv files
# for file in files:
#     if not file.endswith("csv"):
#         files.remove(file)

In [1217]:
# # create empty list 
# df_list = []

# # append datasets to the list
# for file in files:
#     temp_df = pd.read_csv(path + file, sep=',')
#     df_list.append(temp_df)

# # show results
# products, orders, orderlines, brands = df_list[0], df_list[1], df_list[2], df_list[3]

## Data quality

### Missing values

In [1218]:
# we can check missing values column
orderlines.isna().sum()

id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64

In [1219]:
# or for the whole dataframe
orderlines.isna().sum().sum()

0

In [1220]:
# the .info() method also tells us the "Non-Null Count" for each column
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [1221]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


In [1222]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   short   187 non-null    object
 1   long    187 non-null    object
dtypes: object(2)
memory usage: 3.0+ KB


In [1223]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


### Duplicates

The presence of duplicate rows is generally a sign that the data is not clean, and will deserve further exploration.

In [1224]:
orderlines.duplicated().sum()

0

In [1225]:
orders.duplicated().sum()

0

In [1226]:
brands.duplicated().sum()

0

In [1227]:
products.duplicated().sum()

8746

### Data cleanliness - initial assessment

Based on our initial exploration, we know we will need to deal with some missing values. The biggest issue so far are the duplicates on the `products` DataFrame. Here are some aspects that we will need to fix or, at least, explore further:

* **products**: 
    * `price` and `promo_price` are loaded as objects. They should be floats. 
    * Missing values: 
        * `description`: 7 missing values. Maybe that could be inferred from the product name?
        * `price`: the missing values could be filled from the `orderlines` dataset. But first we will need to clean it. 
    * Duplicates: a total of 8746 duplicates seems to indicate this DataFrame has been seriously corrupted.
    
* **orders**: 
    * `created_date` should have a date data type. Then, it would be a good quality check to see if the created date for `orders` mathces with the created dates for `orderlines`. 
    
* **orderlines**: 
    * `unit_price` has to be a float, something is wrong there. 
    * `date` has to be transformed to a date data type. Then, as we said, check that it with matches with the `orders` dataset. 
    
* **brands**: looks fine. 

Where do we have to start? 

1. **Data consistency:** Since `orders` and `orderlines` seem very crucial to the analysis, we will start by cleaning them and checking that the information present in both DataFrames match.

2. **The "products mess":** This file seems to have many issues. We will leave it out for now and perform a proper exploration later to understand better what's going on there.

## Cleaning orders

The data consistency check we will do with `orderlines` will involve two steps: 

* the initial and last dates of the orders should be the same
* the sum of `total_paid` on both datasets should be the same

Let's start by transforming the `created_date` of the orders DataFrame and looking for its earliest and latest values:

In [1228]:
# change date datatype
orders['created_date'] = pd.to_datetime(orders['created_date'])

In [1229]:
# earliest value
min(orders['created_date'])

Timestamp('2017-01-01 00:07:19')

In [1230]:
# latest value
max(orders['created_date'])

Timestamp('2018-03-14 13:58:36')

Now we will look at the overall sum of `total_paid` for the orders table:

In [1231]:
sum(orders['total_paid'])

nan

Why do you think the result of the sum is a nan (not a number)?

In [1232]:
orders.total_paid.isna().sum()

5

There are missing values! We can explore them and see how they are all "Pending" orders:

In [1233]:
orders.loc[orders['total_paid'].isna()]

,order_id,created_date,total_paid,state
127701,427314,2017-11-20 18:54:39,NaN,Pending
132013,431655,2017-11-22 12:15:24,NaN,Pending
147316,447411,2017-11-27 10:32:37,NaN,Pending
148833,448966,2017-11-27 18:54:15,NaN,Pending
149434,449596,2017-11-27 21:52:08,NaN,Pending


Since these orders are only a tiny fraction and there's a valid reason why the `total_paid` value is missing, we will simply exclude them from the dataset:

In [1234]:
orders.dropna(inplace=True)

Now the dataset is clean. And the total paid is: 

In [1235]:
orders['total_paid'].sum()

129159615.07000001

## Cleaning orderlines

Following our data consistency check, will now gather in the orderlines DataFrame the same information we got from orders:

* the initial and last dates
* the sum of `total_paid`


First let's transform our date time:

In [1236]:
orderlines['date'] = pd.to_datetime(orderlines['date'])

In [1237]:
min(orderlines['date'])
# orders: Timestamp('2017-01-01 00:07:19')

Timestamp('2017-01-01 00:07:19')

In [1238]:
max(orderlines['date'])
# orders: Timestamp('2018-03-14 13:58:36')

Timestamp('2018-03-14 13:58:36')

**It's a match!**

Now let's check the `total_paid` for orderlines. It's not going to be as easy as with the orders DataFrame, considering the sturcture of orderlines:


In [1239]:
orderlines.head(3)

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57



To get this value, we will have to calculate a new column, total price for each row. It would be `product_quantity` * `unit_price`. This operation will require that both columns have a numeric data type:

In [1240]:
orderlines.dtypes

id                           int64
id_order                     int64
product_id                   int64
product_quantity             int64
sku                         object
unit_price                  object
date                datetime64[ns]
dtype: object

...it's not the case right now, so we will have to transform the `unit_price` to a numeric data type. 

In [1241]:
# uncomment the line of code below and read the error it produces:
# orderlines['product_quantity'] * pd.to_numeric(orderlines['unit_price'])

While trying to transform this column to numeric an error appears. From the error message, 

> "Unable to parse string '1.137.99' at position 6"

it seems that our dataset has some problems with the thousands separators: they were encoded as dots, and Python & pandas only admit one dot per number: the _decimal_ separator!

Lesson learned: do not use thousand separators in databases / statistical software / programming languages! Sadly, it's too late for us, and we will have to deal with the issue.

There are many ways to approach this problem. The first thing we will do is to count how many dots appear for each `unit_price` value, using string operations. If there are two or more dots a value, we will consider it corrupted —and either try to fix it, or remove it completely.

In [1242]:
# we create a copy of the dataset
ol_temp = orderlines.copy()

# create a new column with the amount of dots in the unit_price column
ol_temp['dots'] = orderlines['unit_price'].str.count('\.') # the backslash 'escapes' the special meaning of '.' in string operations

# show the rows with more than one dot
ol_temp.query('dots > 1')

,id,id_order,product_id,product_quantity,sku,unit_price,date,dots
6,1119115,299544,0,1,APP1582,1.137.99,2017-01-01 01:17:21,2
11,1119126,299549,0,1,PAC0929,2.565.99,2017-01-01 02:07:42,2
15,1119131,299553,0,1,APP1854,3.278.99,2017-01-01 02:14:47,2
43,1119195,299582,0,1,PAC0961,2.616.99,2017-01-01 08:54:00,2
59,1119214,299596,0,1,PAC1599,2.873.99,2017-01-01 09:53:11,2
...,...,...,...,...,...,...,...,...
293862,1649999,452946,0,1,APP2075,2.999.00,2018-03-14 13:03:33,2
293887,1650045,527321,0,1,PAC2148,3.497.00,2018-03-14 13:10:15,2
293889,1650050,527324,0,1,PAC2117,3.075.00,2018-03-14 13:10:56,2
293911,1650088,527342,0,1,APP2492,1.329.00,2018-03-14 13:24:51,2


Our theory about the thousands separators is confirmed. How can we solve this problem? 

Let's remove all the dots for all the `unit_price`, and then add a dot before the last 2 digits to all the rows. Then we will transform it into numeric values.

##### step 1: remove all dots
a) A "corrupted" price like `1.137.99`	will become `113799`

b) A correct price like `37.99`	will become `3799`

##### step 2: add dots two digits before the end of the number
a) The "corrupted" price will go from `113799` to `1137.99`

b) The correct price will go from `3799` back to `37.99`.

In [1243]:
# step 1: to remove the dots, we replace them for... nothing
orderlines = orderlines.assign(unit_price_nd = orderlines['unit_price'].str.replace('\.','', regex=True))
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374


In [1244]:
# step 2.1: we first separate all numbers between the part that goes before the
# decimal point (integers) and the part that goes afterwards (deimals)
orderlines['integers'] = orderlines['unit_price_nd'].str[:-2]
orderlines['decimals'] = orderlines['unit_price_nd'].str[-2:]
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,integers,decimals
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,18,99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,399,00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,474,05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,68,39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,23,74


In [1245]:
# step 2.2: we now concatenate those two parts of the number, with a dot in between
orderlines['new_unit_price'] = orderlines['integers'] + '.' + orderlines['decimals']
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_nd,integers,decimals,new_unit_price
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,1899,18,99,18.99
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,39900,399,00,399.00
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,47405,474,05,474.05
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,6839,68,39,68.39
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2374,23,74,23.74


We will now try again to convert this column to a numeric data type:

In [1246]:
orderlines['unit_price'] = pd.to_numeric(orderlines['new_unit_price'])
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
 7   unit_price_nd     293983 non-null  object        
 8   integers          293983 non-null  object        
 9   decimals          293983 non-null  object        
 10  new_unit_price    293983 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(5)
memory usage: 24.7+ MB


Data cleaning done! 

Back to data consistency: Now it is time to multiply `product_quantity` and `unit price`, sum all the rows and check whether the value matches the sum of the `total_paid` from the orders DataFrame: 

In [1247]:
# drop 'auxiliary' columns
orderlines.drop(['unit_price_nd','decimals','integers','new_unit_price'], axis=1, inplace=True)

# create a new column "total_price" by multiplying product_quantity times unit_price
orderlines['total_price'] = orderlines['product_quantity'] * orderlines['unit_price']

# sum of the new column "total_price":
sum(orderlines['total_price'])

128776222.0298095

Sadly, it does not match exactly with the sum of `total_paid` from orders:

In [1248]:
orders['total_paid'].sum()

129159615.07000001

The mismatch is about 383 thousand dollars, a non-neglegible amount of money:

In [1249]:
sum(orderlines['total_price']) - orders['total_paid'].sum()

-383393.040190503

How can we figure out where does the difference come from? 

## Matching `orders` and `orderlines`

It is possible that some orders exist in one dataset but not in the other one. This would be a potential source for this price mismatch. Let's find out!

We first create a new column in the `orderlines` dataset using `assign`. We also use `isin()` to create a boolen value (True/False) that checks whether the `id_order` is present in the `orders` dataset:

In [1250]:
orderlines.assign(check_orders = orderlines['id_order'].isin(orders['order_id']))

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99,True
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00,True
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05,True
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39,True
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74,True
...,...,...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25,42.99,True
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34,141.58,True
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41,19.98,True
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01,19.99,True


Then, using `.query` we select rows where the value in this new column is `False`:

In [1251]:
(
orderlines
    .assign(check_orders = orderlines['id_order'].isin(orders['order_id']))
    .query("check_orders==False")
)

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
5,1119114,295310,0,10,WDT0249,231.79,2017-01-01 01:14:27,2317.90,False
63,1119218,296284,0,1,BNQ0042,699.00,2017-01-01 09:58:35,699.00,False
67,1119223,294806,0,1,APP1849,2558.99,2017-01-01 10:09:15,2558.99,False
69,1119226,294806,0,1,APP1864,2797.99,2017-01-01 10:15:14,2797.99,False
70,1119235,297261,0,1,QNA0177,304.99,2017-01-01 10:17:59,304.99,False
...,...,...,...,...,...,...,...,...,...
275665,1621177,244328,0,1,OWC0260,349.00,2018-02-20 13:00:45,349.00,False
280055,1628699,261391,0,1,APP2352,3343.00,2018-02-27 11:43:25,3343.00,False
280352,1629247,287797,0,1,SYN0182,484.11,2018-02-27 19:16:26,484.11,False
280856,1630150,261391,0,1,SAT0091,44.99,2018-02-28 12:18:26,44.99,False


It looks like 240 rows in `orderlines` come from orders not present in the `orders` dataset. This is quite inconsistent, since the `orders` dataset should be the one and only source of truth for orders: if an order is not there, it should not exist. We will definitely report this anomaly, but for now, let's just remove those "ghost" orders:

In [1252]:
orderlines = (orderlines
              .assign(check_orders = orderlines['id_order'].isin(orders['order_id']))
              .query("check_orders==True"))

Now let's look at this problem in the opposite direction: are there orders in the `orders` dataset not prsent in `orderlines`?

In [1253]:
(orders
 .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
 .query("check_orders==False"))

,order_id,created_date,total_paid,state,check_orders
8,245941,2017-01-01 10:32:23,183.52,Completed,False
65,268629,2017-01-31 11:27:25,73.23,Completed,False
75,272862,2017-04-18 18:17:10,128.99,Pending,False
82,277655,2017-02-01 08:44:55,9.99,Pending,False
83,277994,2017-01-23 18:30:11,52.99,Completed,False
...,...,...,...,...,...
226835,527328,2018-03-14 13:14:12,0.00,Shopping Basket,False
226851,527344,2018-03-14 13:25:21,0.00,Place Order,False
226853,527346,2018-03-14 13:26:14,0.00,Place Order,False
226855,527348,2018-03-14 13:28:18,0.00,Place Order,False


There are more than 22000 orders in the `orders` dataset that are not present on the `orderlines` dataset!!! We can try to find out why by looking at the state of these orders:

In [1254]:
(orders
 .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
 .query("check_orders==False")
 ['state'].value_counts())

Place Order        12304
Shopping Basket     9810
Completed             45
Cancelled             41
Pending               13
Name: state, dtype: int64

It looks like most of them are orders that were not fully completed: the products were left in the shopping basket or the order was "placed" but maybe not paid (hence the state "Place Order". Some of them were "Completed", though. 

This will require further research, and we might have to come back to these orders if we have to explore consumer behaviour (e.g. why are orders left in the shopping basket?), but for now, for the sake of data consistency, let's drop all of these unmatched orders:

In [1255]:
orders = (orders
          .assign(check_orders = orders['order_id'].isin(orderlines['id_order']))
          .query("check_orders==True")
         )

Let's now check again if the total paid matches:

In [1256]:
orders['total_paid'].sum()

129130877.18

In [1257]:
orderlines['total_price'].sum()

128659393.77

STILL NOT MATCHING!!! And actually, the difference got larger. This is outrageous. Let's keep exploring.

In [1258]:
orderlines['total_price'].sum() - orders['total_paid'].sum()

-471483.4100000113

## Solving the price mismatch

Let's merge both datasets and compare, order by order, the `total_price`. We will call this new merged dataset `orders_info`.

*Note: Remember that the `orderlines` dataset contains one row per product bought: an order where 3 different products were purchased will result in 3 rows there. Therefore, to merge `orderlines` with `orders`, we have to group `orderlines` by its `id_order` and aggregate it by taking the sum of the `total_price`.

In [1259]:
orders_info = (
orderlines
    .groupby('id_order')
    .agg({'total_price':'sum'})
    .merge(orders, how='left', left_on='id_order', right_on='order_id')
    .copy()
)
orders_info

,total_price,order_id,created_date,total_paid,state,check_orders
0,44.99,241319,2017-01-02 13:35:40,44.99,Cancelled,True
1,129.16,241423,2017-11-06 13:10:02,136.15,Completed,True
2,10.77,242832,2017-12-31 17:40:03,15.76,Completed,True
3,77.99,243330,2017-02-16 10:59:38,84.98,Completed,True
4,153.87,243784,2017-11-24 13:35:19,157.86,Cancelled,True
...,...,...,...,...,...,...
204686,42.99,527397,2018-03-14 13:56:38,42.99,Place Order,True
204687,42.99,527398,2018-03-14 13:57:25,42.99,Shopping Basket,True
204688,141.58,527399,2018-03-14 13:57:34,141.58,Shopping Basket,True
204689,19.98,527400,2018-03-14 13:57:41,19.98,Shopping Basket,True


Now that the `total` from both datasets is in the same dataframe, we can create a new column with the difference:

In [1260]:
orders_info['price_difference'] = orders_info['total_price'] - orders_info['total_paid']
orders_info.sort_values('price_difference').tail(30)

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
20654,164.90,322147,2017-02-10 11:56:59,125.99,Completed,True,38.91
11725,164.90,312311,2017-01-20 00:59:54,125.99,Completed,True,38.91
45940,44.99,349377,2017-04-24 21:42:28,4.99,Pending,True,40.00
8545,164.90,308825,2017-01-13 14:27:46,123.99,Completed,True,40.91
13784,164.90,314570,2017-01-25 09:01:35,122.99,Completed,True,41.91
21347,164.90,322901,2017-02-12 13:25:12,122.99,Place Order,True,41.91
25517,164.90,327363,2017-02-21 22:42:17,122.99,Completed,True,41.91
986,164.90,300474,2017-01-02 12:24:54,119.00,Completed,True,45.90
166458,214768.02,484334,2018-01-07 15:58:26,214714.31,Shopping Basket,True,53.71
54853,13450.87,359083,2017-05-25 13:58:31,13391.86,Shopping Basket,True,59.01


In [1261]:
orders_info['price_difference'].describe()

count    204691.000000
mean         -2.303391
std         297.461827
min      -90898.610000
25%          -0.010000
50%           0.000000
75%           0.000000
max       36159.900000
Name: price_difference, dtype: float64

Looks like the maximum and minimun price differences are huge: some orders are really corrupted. But we also see from the mean and the quartiles that the price difference is neglegible for most orders.

## Challenge: Remove outliers

Decide on a criteria for removing orders whenever you cannot trust the price difference between `orders` and `orderlines`. 

Note: this solution does not completely achieve 100% trustable data, but the objective here is to end up with the best possible data with a limited amount of time, which makes a complete revision of the database protocols and the data pipelines is not feasable - and business questions are pending. Documenting and reporting this data cleaning process, including the criteria that you will use for determining what do you consider an "outlier", is key. Not being paralized by it is also important!

In [1262]:
desc = orders_info["price_difference"].describe().reset_index()
mean = desc["price_difference"][1]
std = desc["price_difference"][2]
outlier_upper = mean + 3 * std
outlier_lower = mean - 3 * std

In [1263]:
orders_info[orders_info.price_difference > outlier_upper].sort_values("price_difference")

,total_price,order_id,created_date,total_paid,state,check_orders,price_difference
186388,1124.00,506820,2018-02-06 13:34:20,1.33,Shopping Basket,True,1122.67
179022,1558.00,498550,2018-01-25 17:57:06,3.81,Shopping Basket,True,1554.19
162308,1958.36,479621,2018-01-02 13:33:37,26.38,Shopping Basket,True,1931.98
201739,1968.00,524152,2018-03-09 21:00:48,6.59,Shopping Basket,True,1961.41
192913,2342.00,514198,2018-02-19 11:40:55,6.59,Shopping Basket,True,2335.41
193911,3038.00,515325,2018-02-21 11:31:20,13.19,Shopping Basket,True,3024.81
188169,3112.59,508818,2018-02-09 13:43:30,6.59,Shopping Basket,True,3106.00
197889,7771.23,519801,2018-03-02 08:59:34,31.22,Shopping Basket,True,7740.01
56804,250802.37,361200,2017-06-01 11:55:02,214642.47,Shopping Basket,True,36159.90


In [1264]:
upper_id = orders_info["order_id"][orders_info.price_difference > outlier_upper].to_list()
upper_id

[361200, 479621, 498550, 506820, 508818, 514198, 515325, 519801, 524152]

In [1265]:
lower_id = orders_info["order_id"][orders_info.price_difference < outlier_lower].to_list()
lower_id

[293308,
 297148,
 311490,
 313023,
 334803,
 349166,
 376851,
 431635,
 508825,
 512894]

In [1266]:
orderlines = orderlines[~orderlines.id_order.isin(upper_id)] 
orderlines = orderlines[~orderlines.id_order.isin(lower_id)]

In [1267]:
orders = orders[~orders.order_id.isin(upper_id)]
orders = orders[~orders.order_id.isin(lower_id)]

Save the data once you are sure you can trust it!

In [1268]:
# run the code only once your "orderlines" and "orders" are clean
# orderlines.to_csv('orderlines_clean.csv', index = False)
# orders.to_csv('orders_clean.csv', index = False)

# from google.colab import files
# files.download("orderlines_clean.csv")
# files.download("orders_clean.csv")

## Challenge: Cleaning products

Now it is time to clean the products dataset. Let's do a quick review of its major problems: 

In [1269]:
print(products.info(), "\n")
print("Missing values:", products.isna().sum(), "\n")
print("Duplicate rows:", products.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
None 

Missing values: sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64 

Duplicate rows: 8746


Looking at this overview, we can see that there are different things that have to be changed: 

* Data types: 
    * `price` should be a float
    * `promo price` should be a float
* Duplicated rows. They have to be removed. 
    * To accomplish this step you can use the method `pd.DataFrame.drop_duplicates()`. Be sure you drop all the duplicates based on the column **sku**, as it is the one you will use to merge with orderlines. 
* Missing values: 
    * Description maybe can be inferred by the name
    * `price`. Is there a way we can extract the information from another table?
    * `type`. Do we need this column for our analysis?
    
This task can be accomoplished by using all the methods you already know.

### Start of the challenge

#### Duplicates

In [1290]:
products.drop_duplicates(subset = ["sku"], inplace = True)
products.sample(10)

,sku,name,desc,price,promo_price,in_stock,type,num_dots,price_int,price_dec,new_price,nd_promo,new_promo
6439,PAC1893,Crucial SSD expansion kit MX300 525GB + 8GB 13...,525GB + SSD expansion 8GB 1333Mhz RAM for Mac ...,321.63,2.895.845,0,1433,1,321,63,321.63,2895845,289.58
6593,CRU0050,Crucial MX300 275GB SSD Disk,SSD 275GB SATA Hard Disk 3.0 (6Gb / s) for Mac...,98.39,909.944,0,12215397,1,98,39,98.39,909944,90.99
3144,TUC0232,Housing Flex Elektro Tucano iPhone 6 / 6S Plus...,Cover for iPhone 6 / 6S Plus.,19.9,159.902,0,11865403,1,19,90,19.90,159902,15.99
10326,NOM0020-A,Open - Nomad Vulcanized LSR Silicone Strap Str...,Reconditioned silicone strap matte black for A...,39.99,273.333,0,2449,1,39,99,39.99,273333,27.33
2195,OWC0150,OWC Thunderbolt Cable 2 10m Black,Thunderbolt Cable 2 10m,294.99,2.379.889,0,1325,1,294,99,294.99,2379889,237.98
2296,ADN0030,Adonit Jot Script 2 Evernote Pointer,Crosshair fine tip for cases.,74.99,69.99,0,1229,1,74,99,74.99,6999,69.99
3460,PAC1326,Pack QNAP TS-253A | 8GB RAM | WD 6TB Network,QNAP NAS Pack + 8GB + RAM memory Hard Drive 6T...,801.29,6.521.791,0,12175397,1,801,29,801.29,6521791,652.17
2522,LIF0079,Lifeproof LifeActiv Bike Rack + Adapter QuickM...,Bike Mount for iPhone.,39.99,269.903,1,1216,1,39,99,39.99,269903,26.99
2430,APP1091,Apple iPod Touch 16GB Rosa,New 6th generation iPod Touch 16GB with 8 mega...,229,2.328.113,0,11821715,0,229,00,229.00,2328113,232.81
9142,PAC2430,Synology DS918 NAS Server | 8GB RAM | 12TB (4x...,NAS server of the Plus Series for companies se...,1196.94,10.093.675,0,12175397,1,1196,94,1196.94,10093675,1009.36


#### Fix data types

In [1271]:
missing_price_sku = products[products.price.isna()]["sku"].to_list()
price_lookup = orderlines[orderlines.sku.isin(missing_price_sku)]
price_lookup.drop_duplicates(subset = "sku", inplace = True)
price_lookup.rename(columns = {"unit_price": "price"}, inplace = True)
price_lookup["price"] = price_lookup.price.astype(str)
price_lookup = price_lookup[["sku", "price"]]
price_lookup = price_lookup.set_index("sku")["price"].to_dict()
products["price"] = products["price"].fillna(products.sku.map(price_lookup))
products.dropna(subset = "price", inplace = True)
products.isna().sum()

sku             0
name            0
desc            7
price           0
promo_price     0
in_stock        0
type           50
dtype: int64

In [1272]:
products.sample(10)

,sku,name,desc,price,promo_price,in_stock,type
15541,NTE0048-A,Open - NewerTech NuGuard KX iPad Air Case Black,IPad case ultra resistant to extreme condition...,79.99,177.425,0,1298
13245,MUV0183,Muvit iPhone Folio 7 / 6s / 6 White / Transparent,Soft polyurethane cover with lid for iPhone 7 ...,13.95,89.903,0,11865403
17856,PAC2351,Synology DS718 + NAS Server | 16GB RAM | 20TB ...,Scalable NAS server with transcoding 4K: 4-cor...,1520.71,13.311.791,0,12175397
1874,MYK0012,MyKronoz ZeWatch2 Bluetooth Watch White,SmartWatch monitoring activity and sleep with ...,79,74.899,0,11905404
11475,PIE0016,Part load module connection and micro iPhone 4...,Load module connection parts and original micr...,19.99,199.892,0,21485407
1754,LEP0024,Lepow Poki 5000mAh External Battery USB Green ...,external battery USB dual mode charge small an...,59.99,349.932,0,1515
12636,SPE0172,"Speck SeeThru Case Macbook Pro 13 ""Rosa",Protective polycarbonate shell for MacBook Pro...,49.9,399.905,0,13835403
14965,SHE0039-A,(Open) SwitchEasy iPhone Case 0.35 7 Plus,Ultra Thin Case for iPhone 7 Plus in White,9.99,24.073,0,1298
13193,PAC1511,Pack QNAP TS-251A NAS Server | 8GB RAM | WD 4T...,NAS with 8GB RAM and two hard disks 2TB (2x2TB...,6.336.689,5.609.899,0,12175397
17667,PAC2195,"Second hand - Apple iMac 20 ""Core 2 Duo 226GHz...",Refurbished iMac 20 inch Core 2 Duo | 2GB RAM ...,1199,2.555.945,0,"5,43E+15"


In [1273]:
products["num_dots"] = products.price.str.count(r"\.")
two_dots_sku = products[products.num_dots >= 2]["sku"].to_list()
dots_lookup = orderlines[orderlines.sku.isin(two_dots_sku)]
dots_lookup.drop_duplicates(subset = "sku", inplace = True)
dots_lookup.rename(columns = {"unit_price": "price"}, inplace = True)
dots_lookup["price"] = dots_lookup.price.astype(str)
dots_lookup = dots_lookup[["sku", "price"]]
dots_lookup.sample(10)

,sku,price
281139,UBI0008,129.99
164222,PAC2006,2075.24
71897,QNA0183,2042.99
14038,PAC0562,89.99
5125,APP1477,489.0
102730,REP0369,69.99
13150,DLK0132,142.9
151377,PAC1971,2970.18
5514,PAC1525,1458.99
75591,PAC1480,892.99


In [1274]:
products.set_index("sku", inplace = True)

In [1275]:
products.update(dots_lookup.set_index("sku"))

In [1276]:
products.reset_index(inplace = True)

In [1277]:
products["num_dots"] = products.price.str.count(r"\.")
products["num_dots"].value_counts()
products = products[products.num_dots < 2]

In [1278]:
products[["price_int", "price_dec"]] = products.price.str.split(r"\.", expand = True)

In [1279]:
products["price_dec"] = products.price_dec.fillna(value = "00")

In [1281]:
products["price_dec"] = products.price_dec.apply(lambda x: x + "0" if len(x) == 1 else x[0:2])

In [1283]:
products["new_price"] = products["price_int"] + "." + products["price_dec"]

In [1284]:
products["nd_promo"] = products.promo_price.str.replace(r"\.", "")

In [1285]:
products["new_promo"] = products.apply(lambda row: row["nd_promo"][0: (len(row["new_price"]) - 1)], axis = 1)

In [1286]:
products["new_promo"] = products.new_promo.apply(lambda x: x[0:-2] + "." + x[-2:])

In [1291]:
products["price"] = pd.to_numeric(products.new_price)

In [1292]:
products["promo_price"] = pd.to_numeric(products.new_promo)

In [1293]:
products.drop(["num_dots", "price_int", "price_dec", "new_price", "nd_promo", "new_promo"], axis = 1, inplace = True)

In [1294]:
products.dtypes

sku             object
name            object
desc            object
price          float64
promo_price    float64
in_stock         int64
type            object
dtype: object

In [1295]:
products.shape

(10386, 7)

In [1296]:
products["promo_price"] = products.apply(lambda row: row["promo_price"] * 0.1 if row["price"] < (row["promo_price"] / 1.5) else row["promo_price"], axis = 1)

In [1297]:
products[products.price < products.promo_price]

,sku,name,desc,price,promo_price,in_stock,type
15,MOS0021,Clearguard Moshi MacBook Pro and Air,Keyboard Protector MacBook Pro 13-inch Retina ...,24.95,24.98,0,13835403
39,JMO0026,Just Mobile Lazy Couch Support Mac and iPad,Mac and iPad small lift stand.,19.95,19.98,0,8696
109,APP0574,Apple MacBook Pro 133 '' i7 29GHz | 4GB RAM | ...,Apple MacBook Pro 133 inches (MD101Y / A) with...,1379.00,1385.58,0,1282
167,OWC0037,"OWC Mercury Elite Pro Mini aluminum box 25 ""FW...",outer case 25 inch SATA eSATA / FW800 / FW400 ...,69.99,74.89,0,12995397
251,KIN0074,Kingston DataTraveler SE9 8GB USB 2.0 key,8GB USB 2.0 key minimalist design.,4.99,5.78,0,57445397
...,...,...,...,...,...,...,...
10188,CAV0008,Cavus Support Wall Play 1 White,Support for speaker Play 1 Sonos,29.00,35.09,1,5398
10193,CAV0009,Cavus Foot Support Sonos Play 1 Black,Floor stand for Speaker Sonos Play 1,59.00,71.39,1,5398
10194,CAV0010,Cavus Foot Support Sonos Play 1 White,Floor stand for Speaker Sonos Play 1,59.00,71.39,0,5398
10314,MOP0108,Mophie Powerstation Mini Universal 4000mAh Bat...,external battery capacity 4000mAh output volta...,69.95,79.89,0,1515


In [1298]:
missing_products = orderlines[~orderlines.sku.isin(products.sku)]
missing_products = missing_products["id_order"]
missing_products

117       299638
192       299706
198       299712
253       299766
285       299799
           ...  
263415    491926
263416    491926
263418    491926
273122    510703
276811    515700
Name: id_order, Length: 1414, dtype: int64

In [1299]:
orders[orders.order_id.isin(missing_products)]
orders = orders[~orders.order_id.isin(missing_products)]

In [1300]:
orderlines = orderlines[orderlines.sku.isin(products.sku)]

##### Deal with the "price" column

#### Missing values

In [1301]:
products.type.isna().sum()

48

In [1302]:
products["type"] = products.type.fillna(value = "0")

#### Save & Download the clean products dataset

In [1303]:
products.to_csv("products_clean.csv", index = False)

In [1304]:
orderlines.to_csv("orderlines_clean.csv", index = False)
orders.to_csv("orders_clean.csv", index = False)

## Brands

The brands csv looks fine, so we can work together with it. 

In [1305]:
brands.to_csv("brands_clean.csv", index = False)
# files.download("brands_clean.csv")